In [2]:
# Standard pacakges 
import torch 
from torch import nn, utils
import networkx as nx 
import numpy as np 
import matplotlib.pyplot as plt 

# Pytroch Geometric 
from torch_geometric import utils as gutils
from torch_geometric import nn as gnn # import layers 
from torch_geometric.datasets import Planetoid # import dataset CORA 


In [3]:
def loader_cora_torch(filepath="../data/raw/Planetoid", transform=None, batch_size=1, shuffle=False, device='cuda:0' if torch.cuda.is_available() else 'mps'):
    """Return the CORA dataset"""
    dataset = Planetoid(root=filepath, name='Cora', split='public', num_train_per_class=20, num_val=500, num_test=1000, transform=transform) # return a class of datasets
    data = dataset[0]
    # print some dataset statistics 
    print(f'Loads Cora dataset, at root location: {filepath}')
    print(f'Number of nodes: {data.num_nodes}')
    print(f'Number of edges: {data.num_edges}')
    print(f'Average node degree: {data.num_edges / data.num_nodes:.2f}')
    print(f'Number of training nodes: {data.train_mask.sum()}')
    print(f'Training node label rate: {int(data.train_mask.sum()) / data.num_nodes:.2f}')
    print(f'Has isolated nodes: {data.has_isolated_nodes()}')
    print(f'Has self-loops: {data.has_self_loops()}')
    print(f'Is undirected: {data.is_undirected()}')

    return data.edge_index.to(device), data.x.to(device), data.y.to(device)


# edge_index, node_features, labels = loader_cora_torch(shuffle=True) # load the cora dataset

In [14]:
class GCNConvCustom(nn.Module):
    def __init__(self, 
                edge_index,
                # node_batch,
                input_dim, 
                output_dim, 
                scale=1, 
                random_init=True, 
                with_bias=True,
                device='cuda:0' if torch.cuda.is_available() else 'mps'):
        super(GCNConvCustom, self).__init__()
        # print("layer initialized")

        """Metadata"""
        self.device = device # initialize the hosting device
        self.with_bias = with_bias
        self.scale = scale

        """Calculate Matrices"""
        # the adjacency matrix with self-loop 
        
        self.A = gutils.to_dense_adj(edge_index).to(self.device)[0]
        self.A_self = self.A + torch.diag(torch.ones(self.A.shape[0], device=self.device))
        # print("Adj Matrix with self loop: ", self.A)
        
        # calculate the degree matrix with A after added self loop
        self.D = torch.sum(self.A_self, dim=0).to(self.device)  # Note: these are the elements along the diagonal of D
        # print("Degree Matrix: ", self.D)

        # for diagonal matrix, raising it to any power is the same as raising its diagonal elements to that power
        # we can just apply the -1/2 power to all element of this degree matrix 
        # self.D_half_norm = torch.reciprocal(torch.sqrt(self.D)) 
        # self.D_half_norm = torch.from_numpy(fractional_matrix_power(self.D, -0.5)).to(self.device)
        self.D_half_norm = torch.diag(torch.pow(self.D, -0.5))
        # print("Normalization Matrix: ", self.D_half_norm)

        # normalized adjacency matrix
        # self.A_s = torch.mm(torch.mm(self.D_half_norm, self.A), self.D_half_norm) 
        self.A_s = self.D_half_norm @ self.A_self @ self.D_half_norm
        self.A_s = self.A_s.to(self.device)
        # print(self.A_s.shape)
        
        # initialize learnable weights
        # the weight should have shape of (N , F) where N is the size of the input, and F is the output dimension
        self.W, self.b = None, None
        if random_init: 
            
            self.W = torch.nn.Parameter(
                data=(2 * torch.rand(input_dim, output_dim, device=self.device)-1)*self.scale,  
                requires_grad=True
            )
            # create trainable a bias term for the layer
            self.b = torch.nn.Parameter(
                data=(2 * torch.rand(output_dim, 1, device=self.device)-1)*self.scale,
                requires_grad=True
            )
        else: 
            self.W = torch.nn.Parameter(
                data=torch.zeros(input_dim, output_dim, device=self.device), 
                requires_grad=True
            )
            self.b = torch.nn.Parameter(
                data=torch.zeros(output_dim, 1, device=self.device), 
                requires_grad=True
            )

    def forward(self, H):
        if self.with_bias: 
            return self.A_s @ (H @ self.W) + self.b.T
        else: 
            return self.A_s @ (H @ self.W)

    def get_adj_matrix(self, with_self=False):
        if with_self: 
            return self.A_self 
        return self.A
    
    def get_normalized_adj_matrix(self): 
        return self.A_s

    def get_degree_matrix(self, normalization=False): 
        if normalization: 
            return self.D_half_norm
        return self.D


class GCN_AE(nn.Module): 
    """
    Graph Convolutional Auto-Encoder

    GCN layer implementation from: 
        https://pytorch-geometric.readthedocs.io/en/latest/modules/nn.html#torch_geometric.nn.conv.GCNConv 
    """
    def __init__(self, 
                edge_index, 
                input_size, 
                hidden_size_1,
                hidden_size_2,
                encoding_size,
                random_init = True, 
                with_bias = True,
                device= 'cuda:0' if torch.cuda.is_available() else 'mps'): 
        super().__init__()
        # meta information
        self.device = device
        self.edge_index = edge_index # same as edge list, replace adjacency matrix 
        self.input_size = input_size
        self.hidden_size_1 = hidden_size_1
        self.hidden_size_2 = hidden_size_2
        self.encoding_size = encoding_size
        self.random_init = random_init
        self.with_bias = with_bias

        # training utilities 
        # self.criterion = None 
        # self.optimizer = None 
        
        # layers 
        self.GCN_1 = GCNConvCustom(edge_index=self.edge_index, input_dim=self.input_size, output_dim=self.hidden_size_1, random_init=self.random_init, with_bias=self.with_bias, device=self.device) 
        self.GCN_2 = GCNConvCustom(edge_index=self.edge_index, input_dim=self.hidden_size_1, output_dim=self.encoding_size, random_init=self.random_init, with_bias=self.with_bias, device=self.device)
        # self.FC = nn.Linear(in_features=self.hidden_size_2, out_features=self.encoding_size, device=self.device)

        # activations 
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def encoder(self, X): 
        X_hat = self.GCN_1(X) # first layer: lower dimension feature matrix
        # X_hat = self.relu(X_hat) 
        H = self.GCN_2(X_hat) # second layer: mean matrix
        # Z = self.relu(H)
        # Z = self.FC(H)
        # Z = self.relu(Z) # this activation may or may not be here, doesn't make a difference  
        return H

    def decoder(self, Z): 
        Y_inner = torch.mm(Z, Z.T) # calculate inner product of matrix 
        # Y_inner = Y_inner.reshape((-1)) # flatten the tensor 
        Y = self.sigmoid(Y_inner) # apply activation 
        return Y

    def forward(self, X): 
        Z = self.encoder(X)
        output = self.decoder(Z)
        return output

In [15]:
# data preparation 
"""
Strategy: 
* Randomly sample 50% edges (training), 20% edges (validation), and 30% edges (testing). 
* We want to evaluate against the edges that are uniformly distributed connected to each nodes, 
    so create a couple of random sampled train, validation, test datasets, evaluate the performance against each. 
"""
def extract_train_test_data(edge_index, node_features=None, labels=None, device='cuda:0' if torch.cuda.is_available() else 'mps'): 
    """
    Parameters
        edge_index: torch.Tensor, shape should be (2, # of edges in graph)
        node_features: troch.Tensor, shape should be (# of nodes, feature dimension)
        labels: torch.Tensor, shape should eb (# of nodes, )
        
    Return
        datasets: list, contain randomly sampled (train_matrix, validation_matrix, test_matix) dataset; same length as the number of trials specified. 
            train_matrix: sub-graph created by sampling 50% of the edges from the original graph 
            validation_matix: sub-graph created by sampling 20% of the edges from the original graph 
            test_matrix: sub-graph of the rest of the 30% of the original graph
    """
    num_edges = edge_index.shape[1] 
    train_size, val_size = int(0.5*num_edges), int(0.2*num_edges)
    test_size = num_edges - train_size - val_size
    print(f"""
            There are a total of {num_edges}. \n
            Train dataset has {train_size} edges. \n
            Validation dataset has {val_size} edges. \n
            Test dataset has {test_size} edges. 
          """)
   
    randomize_indices = torch.randperm(num_edges)
    train_indices, val_indices, test_indices = randomize_indices[:train_size], randomize_indices[train_size:train_size+val_size], randomize_indices[train_size+val_size:]
    train_matrix, validation_matrix, test_matrix = edge_index[:, train_indices], edge_index[:, val_indices], edge_index[:, test_indices] 
   
    return train_matrix.to(device), validation_matrix.to(device), test_matrix.to(device)
    
edge_index, node_features, labels = loader_cora_torch(shuffle=True, device='cuda:0' if torch.cuda.is_available() else 'mps') # load the cora dataset
train_subset, validation_subset, test_subset = extract_train_test_data(edge_index)

Loads Cora dataset, at root location: ../data/raw/Planetoid
Number of nodes: 2708
Number of edges: 10556
Average node degree: 3.90
Number of training nodes: 140
Training node label rate: 0.05
Has isolated nodes: False
Has self-loops: False
Is undirected: True

            There are a total of 10556. 

            Train dataset has 5278 edges. 

            Validation dataset has 2111 edges. 

            Test dataset has 3167 edges. 
          


In [19]:
model = GCN_AE(edge_index=train_subset, input_size=node_features.shape[1], hidden_size_1=1000, hidden_size_2=200, encoding_size=100, random_init=True, with_bias=False)
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)


# format train_subset (edge list) to train_matrix (adjacency matrix)
train_matrix = gutils.to_dense_adj(train_subset)[0]

In [31]:
# train model 
from tqdm.notebook import tqdm

epochs = 1500
train_running_loss = 0.0
with tqdm(range(epochs), unit="batch") as tepoch:
    for epoch in tepoch: 
        # train 
        model.train(True)
        optimizer.zero_grad()

        # get output and run backprop 
        output = model(node_features)
        loss = criterion(output, train_matrix)
        loss.backward()
        optimizer.step()

        # calculate accuracy 
        # print(output)
        accuracy = torch.mean(output == train_matrix, dtype=torch.float).item()

        # print results 
        train_running_loss += loss
        tepoch.set_postfix(loss=loss.item(), accuracy=100. * accuracy)

print("Final Training Loss: ", (train_running_loss/epochs).item())

  0%|          | 0/1500 [00:00<?, ?batch/s]

Final Training Loss:  17.638368606567383


In [1]:
# Function to save the model
def saveModel():
    path = "./GCNs/GCN_AE_linkprediction.pth"
    torch.save(model.state_dict(), path)